In [1]:
import re, json
import pandas as pd
import pickle
import networkx as nx

In [2]:
file1 = open('1651385533.result/1651385533.fas.1.clstr.sorted', 'r')
Lines = file1.readlines()

In [3]:
LineIndex = 0
cd_hit_dict = {}
while LineIndex < len(Lines):
    FF = re.search(r'^(>Cluster.*)', Lines[LineIndex])
    if FF != None:
        LineIndex += 1
        li = []
        dkey = ''
        while LineIndex < len(Lines):
            KK = re.search(r'^(.*\*)', Lines[LineIndex])
            if KK != None:
                dkey = KK.group()
                dkey = re.sub(r'.*>','',dkey)
                dkey = re.sub(r'\.\.\. \*','',dkey)
            else:
                KK = re.search(r'^(.*\.\.\.)', Lines[LineIndex])
                dval = KK.group()
                dval = re.sub(r'.*>','',dval)
                dval = re.sub(r'\.\.\.','',dval)
                li.append(dval)
            if(LineIndex == len(Lines)-1):
                break
            FF = re.search(r'^(>Cluster.*)', Lines[LineIndex + 1])
            if (FF == None):
                LineIndex += 1
            else:
                break
        cd_hit_dict.update({dkey: li})
    LineIndex += 1

In [4]:
li = []
for k in cd_hit_dict.keys():
    if(k.startswith('P')):
        if(not(cd_hit_dict[k])):
            li.append(k)
        elif(all(protein.startswith('P') for protein in cd_hit_dict[k])):
            li.append(k)
            li.extend(cd_hit_dict[k])
len(li)

2422

In [5]:
f = open ('positive.json', "r")
 
p_data = json.load(f)

f.close()

In [6]:
pA = []
pB = []
pgr = nx.Graph()
for i in range(len(p_data)):
    pA.append(p_data[i]['pA'])
    pB.append(p_data[i]['pB'])
    pgr.add_edge(p_data[i]['pA'], p_data[i]['pB'])

In [7]:
to_remove = list(set(list(pgr.nodes())) - set(li))

In [8]:
for i in range(len(to_remove)):
    pgr.remove_node(to_remove[i])

In [9]:
f = open ('seq.json', "r")
 
seq_dict = json.load(f)

f.close()

In [10]:
seq_selected = {x : seq_dict[x] for x in li}

In [11]:
nx.to_pandas_edgelist(pgr)

,source,target
0,P0,P1729
1,P0,P1008
2,P0,P209
3,P0,P1730
4,P0,P618
...,...,...
3398,P1717,P2623
3399,P1723,P2625
3400,P1723,P1724
3401,P1725,P1727


In [12]:
with open('celeganslr.pkl', 'wb') as f:
    pickle.dump([nx.to_pandas_edgelist(pgr), seq_dict], f)